In [1]:
import pandas as pd
import numpy as np
import os
import calendar
from pathlib import Path # to interact with file system.
import networkx as nx
from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns # Making statistical graphs on top of what matplot can do 
from sklearn import preprocessing
from sklearn.metrics import jaccard_score
import matplotlib.pylab as plt # %matplotlib inline renders the figure in a notebook

In [4]:
df = pd.read_csv("dataBAN620-finalproj.csv")
df= df[['Indicator','ParentLocationCode','SpatialDimValueCode', 'Location','Period','Dim1','Dim2','FactValueNumeric']]
ten_economies=['China','United States of America','India','Japan','Germany','Russian Federation','Brazil','Indonesia','United Kingdom of Great Britain and Northern Ireland','France','Mexico']
df=df[df['Location'].isin(ten_economies)]
df.columns = [s.strip().replace(' ', '_') for s in df.columns]
df=df.rename(columns={'Dim1':'Sex','Dim2':'Age_Group','ParentLocationCode':'Region_Code'})
df=df.sort_values(by=['Location','Period','Age_Group','Sex'])
df.head()

,Indicator,Region_Code,SpatialDimValueCode,Location,Period,Sex,Age_Group,FactValueNumeric
195954,nMx - age-specific death rate between ages x a...,AMR,BRA,Brazil,2000,Female,1-4 years,0.001
202660,nqx - probability of dying between ages x and x+n,AMR,BRA,Brazil,2000,Female,1-4 years,0.004
214506,lx - number of people left alive at age x,AMR,BRA,Brazil,2000,Female,1-4 years,97402.000
219508,ndx - number of people dying between ages x an...,AMR,BRA,Brazil,2000,Female,1-4 years,392.300
224970,nLx - person-years lived between ages x and x+n,AMR,BRA,Brazil,2000,Female,1-4 years,388666.000


In [6]:
df  = df.reset_index(drop=True)
df2 = df.pivot_table(index=['Region_Code', 'SpatialDimValueCode', 'Location', 'Period', 'Sex', 'Age_Group'],
                             columns='Indicator',
                             values='FactValueNumeric',
                             aggfunc='first').reset_index()
df2.head()

Indicator,Region_Code,SpatialDimValueCode,Location,Period,Sex,Age_Group,Tx - person-years lived above age x,ex - expectation of life at age x,lx - number of people left alive at age x,nLx - person-years lived between ages x and x+n,nMx - age-specific death rate between ages x and x+n,ndx - number of people dying between ages x and x+n,nqx - probability of dying between ages x and x+n
0,AMR,BRA,Brazil,2000,Female,1-4 years,7420561.0,76.18,97402.0,388666.0,0.0010,392.3,0.004
1,AMR,BRA,Brazil,2000,Female,10-14 years,6547182.0,67.58,96875.0,484035.0,0.0003,137.0,0.001
2,AMR,BRA,Brazil,2000,Female,15-19 years,6063147.0,62.68,96739.0,483101.0,0.0005,236.7,0.002
3,AMR,BRA,Brazil,2000,Female,20-24 years,5580047.0,57.82,96502.0,481748.0,0.0006,304.5,0.003
4,AMR,BRA,Brazil,2000,Female,25-29 years,5098299.0,53.00,96197.0,480046.0,0.0008,376.4,0.004


In [8]:
GDP_df=pd.read_csv("national-gdp-penn-world-table.csv")
GDP_df=GDP_df.drop(columns='Entity')
GDP_df= GDP_df.reset_index(drop=True)
GDP_df= GDP_df.rename(columns={"Code":"SpatialDimValueCode","Year":"Period","GDP (output, multiple price benchmarks)":"GDP"})
GDP_df.head()

,SpatialDimValueCode,Period,GDP
0,ALB,1971,6958496300
1,ALB,1972,7246038000
2,ALB,1973,7536605700
3,ALB,1974,7855124000
4,ALB,1975,8164371000


In [11]:
df_merged = pd.merge(df2, GDP_df, on=['SpatialDimValueCode', 'Period'], how='left')
df_merged.head()

,Region_Code,SpatialDimValueCode,Location,Period,Sex,Age_Group,Tx - person-years lived above age x,ex - expectation of life at age x,lx - number of people left alive at age x,nLx - person-years lived between ages x and x+n,nMx - age-specific death rate between ages x and x+n,ndx - number of people dying between ages x and x+n,nqx - probability of dying between ages x and x+n,GDP
0,AMR,BRA,Brazil,2000,Female,1-4 years,7420561.0,76.18,97402.0,388666.0,0.0010,392.3,0.004,1653572400000
1,AMR,BRA,Brazil,2000,Female,10-14 years,6547182.0,67.58,96875.0,484035.0,0.0003,137.0,0.001,1653572400000
2,AMR,BRA,Brazil,2000,Female,15-19 years,6063147.0,62.68,96739.0,483101.0,0.0005,236.7,0.002,1653572400000
3,AMR,BRA,Brazil,2000,Female,20-24 years,5580047.0,57.82,96502.0,481748.0,0.0006,304.5,0.003,1653572400000
4,AMR,BRA,Brazil,2000,Female,25-29 years,5098299.0,53.00,96197.0,480046.0,0.0008,376.4,0.004,1653572400000


In [12]:
nc_df = pd.read_csv("NonCommunicable_Disease_dataset.csv")
nc_df= nc_df[['ParentLocationCode','SpatialDimValueCode','Location','Period','Dim1','Dim2','FactValueNumeric','FactValueNumericLow','FactValueNumericHigh']]

In [17]:
both_sexes_mask = nc_df['Dim1'] == 'Both sexes'

# Create separate DataFrames for 'Male' and 'Female'
df_male = nc_df.copy()
df_female = nc_df.copy()

# Update 'Dim1' column in each DataFrame
df_male.loc[both_sexes_mask, 'Dim1'] = 'Male'
df_female.loc[both_sexes_mask, 'Dim1'] = 'Female'

# Concatenate the three DataFrames
df_result = pd.concat([nc_df[~both_sexes_mask], df_male[both_sexes_mask], df_female[both_sexes_mask]])
df_result = df_result.rename(columns={'Dim1':'Sex','Dim2':'non-communicable_diseases','ParentLocationCode':'Region_Code'})
# Reset index
df_result.reset_index(drop=True, inplace=True)
df_result

,Region_Code,SpatialDimValueCode,Location,Period,Sex,non-communicable_diseases,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
0,SEAR,IND,India,2019,Female,Cardiovascular diseases,1115687.00,821773.00,1436096.0
1,SEAR,IND,India,2019,Male,Cardiovascular diseases,1450806.00,1111032.00,1849975.0
2,WPR,CHN,China,2019,Male,Malignant neoplasms,1659677.00,1277365.00,2086563.0
3,WPR,CHN,China,2019,Female,Cardiovascular diseases,1877096.00,1463568.00,2282684.0
4,AFR,ZWE,Zimbabwe,2019,Female,Cardiovascular diseases,10035.00,6031.00,15524.0
...,...,...,...,...,...,...,...,...,...
58555,AFR,COD,Democratic Republic of the Congo,2000,Female,Diabetes mellitus,9705.00,5899.00,14976.0
58556,EUR,BIH,Bosnia and Herzegovina,2000,Female,Diabetes mellitus,974.70,657.20,1384.0
58557,EUR,MLT,Malta,2000,Female,Diabetes mellitus,98.27,72.79,130.2
58558,EUR,LUX,Luxembourg,2000,Female,Malignant neoplasms,982.60,783.50,1206.0


In [19]:
df_result = df_result.groupby(['Region_Code','SpatialDimValueCode','Location', 'Period','Sex','non-communicable_diseases'], as_index=False)[['FactValueNumeric','FactValueNumericLow','FactValueNumericHigh']].sum()
df_result=df_result[df_result['Location'].isin(ten_economies)]
df_result=df_result.reset_index(drop=True)
df_result

,Region_Code,SpatialDimValueCode,Location,Period,Sex,non-communicable_diseases,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
0,AMR,BRA,Brazil,2000,Female,Cardiovascular diseases,442596.0,386808.0,487343.0
1,AMR,BRA,Brazil,2000,Female,Diabetes mellitus,65949.0,56471.0,75314.0
2,AMR,BRA,Brazil,2000,Female,Malignant neoplasms,207022.0,184424.0,228142.0
3,AMR,BRA,Brazil,2000,Female,Respiratory diseases,111660.0,90305.0,123246.0
4,AMR,BRA,Brazil,2000,Male,Cardiovascular diseases,459255.0,405489.0,504280.0
...,...,...,...,...,...,...,...,...,...
1755,WPR,JPN,Japan,2019,Female,Respiratory diseases,183848.0,111046.0,268100.0
1756,WPR,JPN,Japan,2019,Male,Cardiovascular diseases,528323.0,409896.0,615647.0
1757,WPR,JPN,Japan,2019,Male,Diabetes mellitus,23543.0,16425.0,30587.0
1758,WPR,JPN,Japan,2019,Male,Malignant neoplasms,634851.0,520942.0,713168.0
